In [1]:
from __future__ import print_function

import pandas as pd
import subprocess
import time


In [2]:
try:
    import pyorient
except ImportError as e:
    #process = subprocess.Popen("pip install --user pyorient".split(), stdout=subprocess.PIPE)
    process = subprocess.Popen("pip install git+https://github.com/DBertazioli/pyorient --user".split(), stdout=subprocess.PIPE)

    output, error = process.communicate()
    print(output, "error = {}".format(error), e)
    import pyorient

In [3]:
db="neomeetup1"
my_auth="root"

In [4]:
client = pyorient.OrientDB("10.9.13.4", 7687)
session_id = client.connect(my_auth,my_auth)

In [5]:
#useful funct
def reset_db(client, name):

    # Remove Old Database
    client.db_drop(name)

    # Create New Database
    try:
        client.db_create(
           db,
           pyorient.DB_TYPE_GRAPH,
           pyorient.STORAGE_TYPE_PLOCAL)
       #logging.info("neomeetup1 Database Created.")
    except pyorient.PyOrientException as err:
       #logging.critical(
       #   "Failed to create neomeetup DB: %" 
       #   % err)
        print(err, "\n err while resetting")
        
def _my_callback(for_every_record):
    print(for_every_record)        

In [6]:
if client.db_exists(db):
   # Open Database
    print("opening db: {}".format(db))
    try:
        client.db_open(db, my_auth, my_auth)
    except pyorient.PyOrientException as err:
        print(err)
        
else:
    try:
        client.db_create(
           db,
           pyorient.DB_TYPE_GRAPH,
           pyorient.STORAGE_TYPE_PLOCAL)
       #logging.info("neomeetup1 Database Created.")
    except pyorient.PyOrientException as err:
       #logging.critical(
       #   "Failed to create neomeetup DB: %" 
       #   % err)
        print(err)

opening db: neomeetup1


In [7]:
df=pd.read_csv("../csv/struttura/member.csv")
df.head()

,member_id,member_name
0,220200963,Sarah Roxas
1,253755396,Lupco Trpeski
2,272979286,Malikmika
3,6,Scott Heiferman
4,232783879,Luis Eduardo Saltini


In [8]:
reset_db(client, db)
try:
    client.command( "create class Member extends V" )
except pyorient.PyOrientException as e:
    print(e)

In [9]:
start_t=time.time()
n=10
debug=True
n_start=1
for line in df.itertuples():
    if debug:
        if line.Index > n_start:
            client.command(
       "create VERTEX Member set member_id = {}, member_name = '{}'".format(line.member_id, str(line.member_name).replace("\"", "").replace("'",""))
            )
    else:
        client.command(
       "create VERTEX Member set member_id = {}, member_name = '{}'".format(line.member_id, str(line.member_name).replace("\"", "").replace("'",""))
            )
    if line.Index == n:
        print("reached line {} in {}".format(n, time.time()-start_t))
        break

reached line 10 in 0.3294241428375244


In [10]:
result = client.command("select from Member where member_id = 6 ", 100)
result #lol useless but will find out

In [11]:


result = client.query_async("select from Member", 10, '*:0', _my_callback)

{'@Member':{'member_id': 272979286, 'member_name': 'Malikmika'},'version':1,'rid':'#17:0'}
{'@Member':{'member_id': 260046859, 'member_name': 'Sikares Pholhem'},'version':1,'rid':'#17:1'}
{'@Member':{'member_id': 62914582, 'member_name': 'Lydia W.'},'version':1,'rid':'#17:2'}
{'@Member':{'member_id': 6, 'member_name': 'Scott Heiferman'},'version':1,'rid':'#18:0'}
{'@Member':{'member_id': 278921232, 'member_name': 'diana'},'version':1,'rid':'#18:1'}
{'@Member':{'member_id': 232783879, 'member_name': 'Luis Eduardo Saltini'},'version':1,'rid':'#19:0'}
{'@Member':{'member_id': 277173592, 'member_name': 'Josh Johnson'},'version':1,'rid':'#19:1'}
{'@Member':{'member_id': 138412042, 'member_name': 'Brad'},'version':1,'rid':'#20:0'}
{'@Member':{'member_id': 77594642, 'member_name': 'Stephanie Franco'},'version':1,'rid':'#20:1'}


In [ ]:
debug=True
#debug1=True
debug1=False
n=50000
start_t=time.time()
batch_query=['begin']
for line in df.itertuples():
    if not debug1:
        query="create VERTEX Member set member_id = {}, member_name = '{}'".format(line.member_id, str(line.member_name).replace("\"", "").replace("'",""))
        query.encode("utf-8")
        batch_query.append(query)
    
        if line.Index is not 0:
            if line.Index % n == 0:
                batch_query.append("commit retry 100")
                cmd=';'.join(batch_query)
                print("line is ", line.Index)
                print("committing")
                m_time=time.time()
                client.batch(cmd)
                print("committed in {} s".format((time.time()-m_time)))

                batch_query=['begin']
    else:
        if line.Index >= 360000:
            query="create VERTEX Member set member_id = {}, member_name = '{}'".format(line.member_id, str(line.member_name).replace("\"", "").replace("'",""))
            query.encode("utf-8")
            batch_query.append(query)

            if line.Index % n == 0:
                batch_query.append("commit retry 100")
                cmd=';'.join(batch_query)
                print("line is ", line.Index)
                print("committing")
                m_time=time.time()
                client.batch(cmd)
                print("committed in {} s".format((time.time()-m_time)))
                
                batch_query=['begin']
    if debug:
        if not debug1:
            if line.Index == 2000:
                print("reached line {} in {} s".format(line.Index, (time.time()-start_t)*1000))
                print("breaking")
                break

if not debug:
    print("reached line {} in {} s".format(len(df), (time.time()-start_t)*1000))

#batch_query.append("commit retry 100")
#cmd=';'.join(batch_query)
#cmd

line is  360000
committing
committed in 0.03893327713012695 s
line is  370000
committing
committed in 2.3277628421783447 s
line is  380000
committing
committed in 2.333251714706421 s
line is  390000
committing
committed in 2.1816258430480957 s
line is  400000
committing
committed in 2.3253912925720215 s
line is  410000
committing
committed in 2.2989134788513184 s
line is  420000
committing
committed in 2.218219757080078 s
line is  430000
committing
committed in 2.208942174911499 s
line is  440000
committing
committed in 2.920119524002075 s
line is  450000
committing
committed in 2.2335426807403564 s
line is  460000
committing
committed in 2.8273231983184814 s
line is  470000
committing
committed in 2.208495616912842 s
line is  480000
committing
committed in 9.496097326278687 s
line is  490000
committing
committed in 2.5114426612854004 s
line is  500000
committing
committed in 2.5195000171661377 s
line is  510000
committing
committed in 2.7344155311584473 s
line is  520000
committing
co

In [ ]:
start_t=time.time()
client.batch(cmd)
time.time()-start_t

In [ ]:
len(df)

In [ ]:
df.head(360165).tail(6)